In [4]:
#Baixar biblioteca 
!pip install ffmpeg


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
#Função para criar um csv dos dados de ruído
import os
import pandas as pd
from pydub import AudioSegment

def create_audio_csv(folder_path):
    # Criando uma lista vazia para armazenar as informações dos arquivos
    audio_list = []
    # Iterando sobre os arquivos na pasta
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".wav"):
                audio_path = os.path.join(root, file)
                audio = AudioSegment.from_wav(audio_path)
                duration = len(audio) / 1000 # convertendo para segundos
                audio_list.append({'path':audio_path, 'folder':os.path.basename(root), 'name':file, 'duration':duration})
    # Criando um dataframe a partir da lista de arquivos e salvando em um arquivo CSV
    audio_df = pd.DataFrame(audio_list)
    audio_df.to_csv('csv_ruidos.csv',index=False)

#Exemplo de uso
create_audio_csv('pastaruidos')

In [9]:
#Separando dados de treino
treino = pd.read_csv("metadata/metadata_train.csv")

In [25]:
treino

,wav_file,label,dataset,language,speaker
0,dataset-ser/train/sad/MOSEI/xSCvspXYU9k.wav,sad,MOSEI,english,xxx
1,dataset-ser/train/happy/MOSEI/_UNQDdiAbWI.wav,happy,MOSEI,english,xxx
2,dataset-ser/train/happy/MOSEI/-qDkUB0GgYY.wav,happy,MOSEI,english,xxx
3,dataset-ser/train/happy/MOSEI/zFuRNvfC4So.wav,happy,MOSEI,english,xxx
4,dataset-ser/train/happy/MOSEI/R-HKj8Yn06k.wav,happy,MOSEI,english,xxx
...,...,...,...,...,...
32799,dataset-ser/train/surprise/SAVEE/KL_su09.wav,surprise,SAVEE,english,SAVEE-KL
32800,dataset-ser/train/surprise/SAVEE/KL_su08.wav,surprise,SAVEE,english,SAVEE-KL
32801,dataset-ser/train/surprise/SAVEE/KL_su15.wav,surprise,SAVEE,english,SAVEE-KL
32802,dataset-ser/train/surprise/SAVEE/KL_su14.wav,surprise,SAVEE,english,SAVEE-KL


In [36]:
treino.to_csv('metadata/treino.csv',index=False)

In [38]:
#Duplicando pois alterarei o csv depois para usar com os dados com ruído
treino.to_csv('metadata/treinoruidoso.csv',index=False)

In [18]:
#instalando biblioteca
!pip install pydub


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
#Criando função para fazer a junção dos áudios de fala com os áudios de ruído
import os
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm

def join_audio_csv(audio_csv, noise_csv):
    # Carregando os arquivos CSV
    audio_df = pd.read_csv(audio_csv)
    noise_df = pd.read_csv(noise_csv)

    # Criando a pasta para salvar os áudios combinados
    if not os.path.exists("dadosruidosos"):
        os.makedirs("dadosruidosos")

    # Iterando sobre as linhas dos arquivos CSV com tqdm
    for index, audio_row in tqdm(audio_df.iterrows()):
        # Carregando o áudio
        audio = AudioSegment.from_wav(audio_row['wav_file'])
        noise = AudioSegment.from_wav(noise_df.iloc[index % len(noise_df)]['path'])
        
        # Ajustando o tamanho do áudio de ruído para combinar com o áudio original
        noise = noise.set_channels(1)
        noise = noise.set_frame_rate(audio.frame_rate)
        noise = noise[:len(audio)]
        
        # Combinando os dois áudios
        combined_audio = audio.overlay(noise)
        save_path = 'noise/'+audio_row['wav_file'].split('.')[0] + '_noise.wav'
        combined_audio.export(save_path, format="wav")

#Exemplo de uso
join_audio_csv('metadata/treino.csv', 'csv_ruidos.csv')


19761it [04:57, 66.44it/s] 


In [77]:
#Função para renomear os arquivos do dataset anteriormente duplicado
def rename_csv_column(csv_file):
    df = pd.read_csv(csv_file)
    df['wav_file'] = df['wav_file'].str.replace(".wav", "_noise.wav")
    df.to_csv(csv_file, index=False)

#Exemplo de uso
rename_csv_column("metadata/treinoruidoso.csv")

In [44]:
import pandas as pd
treino_ruidoso = pd.read_csv('metadata/treinoruidoso.csv')

In [89]:
df.to_csv('metadata/treinoruidoso.csv',index=False)

In [13]:
#Fazendo o mesmo processo para os dados de teste
import pandas as pd
df = pd.read_csv('metadata/testeravdess.csv')

In [14]:
df

,Unnamed: 0,wav_file,label,dataset,language,speaker
0,12660,dataset-ser/train/angry/RAVDESS/03-01-05-02-02...,angry,RAVDESS,english,RAVDESS-01
1,12661,dataset-ser/train/angry/RAVDESS/03-01-05-01-01...,angry,RAVDESS,english,RAVDESS-01
2,12662,dataset-ser/train/angry/RAVDESS/03-01-05-02-01...,angry,RAVDESS,english,RAVDESS-01
3,12663,dataset-ser/train/angry/RAVDESS/03-01-05-02-02...,angry,RAVDESS,english,RAVDESS-01
4,12664,dataset-ser/train/angry/RAVDESS/03-01-05-01-02...,angry,RAVDESS,english,RAVDESS-01
...,...,...,...,...,...,...
1139,32739,dataset-ser/train/surprise/RAVDESS/03-01-08-02...,surprise,RAVDESS,english,RAVDESS-24
1140,32740,dataset-ser/train/surprise/RAVDESS/03-01-08-02...,surprise,RAVDESS,english,RAVDESS-24
1141,32741,dataset-ser/train/surprise/RAVDESS/03-01-08-01...,surprise,RAVDESS,english,RAVDESS-24
1142,32742,dataset-ser/train/surprise/RAVDESS/03-01-08-02...,surprise,RAVDESS,english,RAVDESS-24


In [15]:
df.to_csv('metadata/testeravdessruidoso.csv',index=False)

In [12]:
df

,Unnamed: 0,wav_file,label,dataset,language,speaker
0,12660,dataset-ser/train/angry/RAVDESS/noise/03-01-05...,angry,RAVDESS,english,RAVDESS-01
1,12661,dataset-ser/train/angry/RAVDESS/noise/03-01-05...,angry,RAVDESS,english,RAVDESS-01
2,12662,dataset-ser/train/angry/RAVDESS/noise/03-01-05...,angry,RAVDESS,english,RAVDESS-01
3,12663,dataset-ser/train/angry/RAVDESS/noise/03-01-05...,angry,RAVDESS,english,RAVDESS-01
4,12664,dataset-ser/train/angry/RAVDESS/noise/03-01-05...,angry,RAVDESS,english,RAVDESS-01
...,...,...,...,...,...,...
1139,32739,dataset-ser/train/surprise/RAVDESS/noise/03-01...,surprise,RAVDESS,english,RAVDESS-24
1140,32740,dataset-ser/train/surprise/RAVDESS/noise/03-01...,surprise,RAVDESS,english,RAVDESS-24
1141,32741,dataset-ser/train/surprise/RAVDESS/noise/03-01...,surprise,RAVDESS,english,RAVDESS-24
1142,32742,dataset-ser/train/surprise/RAVDESS/noise/03-01...,surprise,RAVDESS,english,RAVDESS-24


In [6]:
import os
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm

def join_audio_csv(audio_csv, noise_csv):
    # Carregando os arquivos CSV
    audio_df = pd.read_csv(audio_csv)
    noise_df = pd.read_csv(noise_csv)

    # Criando a pasta para salvar os áudios combinados
    if not os.path.exists("dadosruidosos"):
        os.makedirs("dadosruidosos")

    # Iterando sobre as linhas dos arquivos CSV com tqdm
    for index, audio_row in tqdm(audio_df.iterrows()):
        # Carregando o áudio
        audio = AudioSegment.from_wav(audio_row['wav_file'])
        noise = AudioSegment.from_wav(noise_df.iloc[index % len(noise_df)]['path'])
        
        # Ajustando o tamanho do áudio de ruído para combinar com o áudio original
        noise = noise.set_channels(1)
        noise = noise.set_frame_rate(audio.frame_rate)
        noise = noise[:len(audio)]
        
        # Combinando os dois áudios
        combined_audio = audio.overlay(noise)
        save_path = audio_row['wav_file'].split('.')[0] + '_noise.wav'
        combined_audio.export(save_path, format="wav")

#Exemplo de uso
join_audio_csv('metadata/testeravdess.csv', 'csv_ruidos.csv')


1144it [00:19, 57.50it/s]


In [16]:
def rename_csv_column(csv_file):
    df = pd.read_csv(csv_file)
    df['wav_file'] = df['wav_file'].str.replace(".wav", "_noise.wav")
    df.to_csv(csv_file, index=False)

#Exemplo de uso
rename_csv_column("metadata/testeravdessruidoso.csv")

C:\Users\heloi\AppData\Local\Temp\ipykernel_14348\1517836639.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['wav_file'] = df['wav_file'].str.replace(".wav", "_noise.wav")
